In [73]:
import urllib.request as ureq
ureq.urlretrieve('https://raw.githubusercontent.com/ugik/notebooks/master/intents.json','intents.json')

('intents.json', <http.client.HTTPMessage at 0x7fadce6ec3d0>)

In [74]:
# things we need for NLP
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
nltk.download('punkt')
# things we need for Tensorflow
import numpy as np
import random

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [75]:
# import our chat-bot intents file
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [76]:
words = []
classes = []
documents = []
ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

27 documents
9 classes ['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']
48 unique stemmed words ["'d", "'s", 'a', 'acceiv', 'anyon', 'ar', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'do', 'doe', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'hour', 'how', 'i', 'is', 'kind', 'lat', 'lik', 'mastercard', 'mop', 'of', 'on', 'op', 'rent', 'see', 'tak', 'thank', 'that', 'ther', 'thi', 'to', 'today', 'we', 'what', 'when', 'which', 'work', 'yo', 'you']


In [77]:
# create our training data
training = []
output = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [78]:
# reset underlying graph data
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
model=Sequential()
model.add(Dense(8,input_shape=(len(train_x[0]),),kernel_initializer='normal'))
model.add(Dense(8,kernel_initializer='normal'))
model.add(Dense(len(train_y[0]),activation='softmax',kernel_initializer='normal'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
model.fit(np.array(train_x),np.array(train_y), epochs=1000, batch_size=8, verbose=1)

Train on 27 samples
Epoch 1/1000
27/27 [==============================] - 0s 3ms/sample - loss: 0.0988 - acc: 0.0000e+00
Epoch 2/1000
27/27 [==============================] - 0s 454us/sample - loss: 0.0987 - acc: 0.2222
Epoch 3/1000
27/27 [==============================] - 0s 485us/sample - loss: 0.0987 - acc: 0.1852
Epoch 4/1000
27/27 [==============================] - 0s 536us/sample - loss: 0.0987 - acc: 0.2222
Epoch 5/1000
27/27 [==============================] - 0s 511us/sample - loss: 0.0987 - acc: 0.2222
Epoch 6/1000
27/27 [==============================] - 0s 451us/sample - loss: 0.0987 - acc: 0.2222
Epoch 7/1000
27/27 [==============================] - 0s 552us/sample - loss: 0.0986 - acc: 0.1852
Epoch 8/1000
27/27 [==============================] - 0s 423us/sample - loss: 0.0986 - acc: 0.1852
Epoch 9/1000
27/27 [==============================] - 0s 473us/sample - loss: 0.0986 - acc: 0.1852
Epoch 10/1000
27/27 [==============================] - 0s 442us/sample - loss: 0.0985 -

In [79]:
def clean_up_sentence(sentence):
    # tokenize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [80]:
p = bow("is your shop open today?", words)
print (p)
print (classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0]
['goodbye', 'greeting', 'hours', 'mopeds', 'opentoday', 'payments', 'rental', 'thanks', 'today']


In [81]:
print(model.predict(np.array([p])))

[[1.00553094e-10 4.79542541e-05 3.67024690e-01 1.44647405e-04
  6.32644415e-01 2.56583590e-08 3.59329679e-05 2.61189209e-10
  1.02356375e-04]]


/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2067: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [82]:
context = {}

ERROR_THRESHOLD = 0.25
def classify(sentence):
    # generate probabilities from the model
    results = model.predict(np.array([bow(sentence, words)]))[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # set context for this intent if necessary
                    if 'context_set' in i:
                        if show_details: print ('context:', i['context_set'])
                        context[userID] = i['context_set']

                    # check if this intent is contextual and applies to this user's conversation
                    if not 'context_filter' in i or \
                        (userID in context and 'context_filter' in i and i['context_filter'] == context[userID]):
                        if show_details: print ('tag:', i['tag'])
                        # a random response from the intent
                        return print(random.choice(i['responses']))

            results.pop(0)

In [83]:
classify('is your shop open today?')

[('opentoday', 0.6326444), ('hours', 0.3670247)]

In [84]:
response('is your shop open today?')

We're open every day from 9am-9pm


In [85]:
response('do you take cash?')

We accept most major credit cards


In [86]:
response('what kind of mopeds do you rent?')

We rent Yamaha, Piaggio and Vespa mopeds


In [87]:
response('Goodbye, see you later')

Bye! Come back again soon.


In [88]:
context

{}

In [89]:
response('we want to rent a moped')

Are you looking to rent today or later this week?


In [90]:
context

{'123': 'rentalday'}

In [91]:
response('today')

For rentals today please call 1-800-MYMOPED


In [92]:
classify('today')

[('today', 0.9719282)]

In [93]:
response("Hi there!", show_details=True)

context: 
tag: greeting
Good to see you again


In [94]:
response('today')
classify('today')

[('today', 0.9719282)]

In [95]:
response("thanks, your great")

Happy to help!
